In [1]:
import time
import random
import selenium

from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains






In [2]:
username = "K.Dixon1@lse.ac.uk"
password = ""

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
import json

def scrape_exam_papers(driver, start_box):
    list = []

    def get_elements(css_selector, retries=3):
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, css_selector)))
            except (StaleElementReferenceException, TimeoutException) as e:
                if attempt < retries - 1:
                    time.sleep(1)
                    continue
                else:
                    print(f"Failed to get elements: {e}")
                    return []

    def get_element(css_selector, retries=3):
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
            except (StaleElementReferenceException, TimeoutException) as e:
                if attempt < retries - 1:
                    time.sleep(1)
                    continue
                else:
                    print(f"Failed to get element: {e}")
                    return None

    def click_load_more():
        try:
            load_more_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div/prm-explore-main/ui-view/prm-collection-discovery/prm-collection-gallery/md-content/div[2]/prm-gallery-items-list/div/div[3]/button")))
            load_more_button.click()
            time.sleep(1)
            print("Clicked 'Load More' button.")
        except (NoSuchElementException, TimeoutException) as e:
            print(f"'Load More' button not found or not clickable: {e}. Continuing...")

    def find_show_more():
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div[3]/div/md-dialog/md-dialog-content/sticky-scroll/prm-full-view/div/div/div/div/div[1]/div[4]/div/prm-full-view-service-container/div[2]/div/prm-alma-viewit/prm-alma-viewit-items/button")))
            return True
        except TimeoutException:
            return False

    def click_show_more():
        try:
            show_more_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div[3]/div/md-dialog/md-dialog-content/sticky-scroll/prm-full-view/div/div/div/div/div[1]/div[4]/div/prm-full-view-service-container/div[2]/div/prm-alma-viewit/prm-alma-viewit-items/button")))
            show_more_button.click()
            time.sleep(1)
            print("Clicked 'Show More' button.")
        except (NoSuchElementException, TimeoutException) as e:
            print(f"'Show More' button not found or not clickable: {e}. Continuing...")

    try:
        driver.execute_script("document.body.style.zoom='30%'")
        boxes = get_elements(".margin-bottom-small > prm-gallery-collection")

        for m in range(start_box, len(boxes)):
            for retry in range(3):  # Retry loop for each box
                try:
                    # Re-find the boxes element to avoid stale reference
                    driver.execute_script("document.body.style.zoom='30%'")
                    boxes = get_elements(".margin-bottom-small > prm-gallery-collection")
                    box = boxes[m]
                    details = {}

                    # Extract title and URL
                    title = get_element(f".margin-bottom-small > prm-gallery-collection:nth-child({m+1}) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(2) > span:nth-child(1)").text
                    subject_page_url = box.find_element(By.TAG_NAME, "a").get_attribute("href")
                    box.click()

                    if m in [1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 19, 23, 24]:
                        while True:
                            click_load_more()
                            try:
                                load_more_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/primo-explore/div/prm-explore-main/ui-view/prm-collection-discovery/prm-collection-gallery/md-content/div[2]/prm-gallery-items-list/div/div[3]/button")))
                                load_more_button.click()
                            except TimeoutException:
                                break

                    containers = get_elements(f".is-grid-view > prm-gallery-item")

                    for i in range(len(containers)):
                        containers = get_elements(f".is-grid-view > prm-gallery-item")
                        driver.execute_script("arguments[0].scrollIntoView();", containers[i])
                        container = containers[i]

                        course_name = container.find_element(By.CLASS_NAME, "item-title").text
                        type = container.find_element(By.CSS_SELECTOR, "div:nth-child(2) > div:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text
                        container.click()

                        exams = get_elements("md-list-item.md-3-line")

                        if find_show_more():
                            click_show_more()
                            time.sleep(1)
                        else:
                            print("'Show More' button not visible. Continuing...")

                        for n in range(1, len(exams) + 1):
                            exam = get_element(f"md-list-item.md-3-line:nth-child({n}) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)")
                            if exam:
                                driver.execute_script("arguments[0].scrollIntoView();", exam)
                                exam_year = exam.text
                                pdf_link = exam.get_attribute("href")

                                details = {
                                    "name": title,
                                    "subject_page_url": subject_page_url,
                                    "course_name": course_name,
                                    "type": type,
                                    "exam_year": exam_year,
                                    "pdf_link": pdf_link
                                }
                                list.append(details)
                                print(details)

                        exit_button = get_element("button.md-icon-button:nth-child(4)")
                        if exit_button:
                            exit_button.click()
                        else:
                            print("Exit button not found. Retrying...")
                            raise TimeoutException("Exit button not found")

                except (StaleElementReferenceException, TimeoutException) as e:
                    print(f"Error processing box {m}: {e}. Retrying...")
                    # Navigate back and retry
                    back_button = get_element("prm-collection-navigation-breadcrumbs-item.layout-align-start-center:nth-child(2) > a:nth-child(1)")
                    if back_button:
                        driver.execute_script("arguments[0].scrollIntoView();", back_button)
                        back_button.click()
                        time.sleep(5)
                        WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".margin-bottom-small > prm-gallery-collection")))
                        continue
                    else:
                        print("Back button not found. Breaking out of retry loop.")
                        break

                else:
                    break  # Break out of retry loop if no exception

            # Save current list to JSON file
            with open(f"exam_papers_{m}.json", "w") as f:
                json.dump(list, f, indent=4)
                print(f"Saved to exam_papers_{m}.json")
            back_button = get_element("prm-collection-navigation-breadcrumbs-item.layout-align-start-center:nth-child(2) > a:nth-child(1)")
            if back_button:
                driver.execute_script("arguments[0].scrollIntoView();", back_button)
                time.sleep(5)
                back_button.click()
                time.sleep(15)
                WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".margin-bottom-small > prm-gallery-collection")))

    except (StaleElementReferenceException, TimeoutException) as e:
        print(f"Container processing error: {e}")

    return list


In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ...

driver = webdriver.Firefox()
driver.get('https://www.lse.ac.uk/library')

# enter full screen
driver.fullscreen_window()

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'Loginto'))).click()

# Let the user actually see something!
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[3]/div/div/h3[1]/a'))).click()

# Let the user actually see something!
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.NAME, 'loginfmt'))).send_keys(username)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
if next_button:
    next_button.click()
else:
    print("No next button found")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.NAME, 'passwd'))).send_keys(password)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
next_button.click()

# Wait until the screen changes, this can be a variable length of time though
try:
    # Wait until the element is visible
    element = WebDriverWait(driver, 300).until(EC.visibility_of_element_located((By.ID, 'idBtn_Back')))
    # Act on the element as soon as it becomes visible
    element.click()
except TimeoutException:
    print("The element did not appear within the time limit")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.grid-item:nth-child(1) > button:nth-child(1)'))).click()

new_url = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)')))[0].get_attribute("href")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)'))).click()

#the driver must now be on the new page

WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
new_window = driver.window_handles[-1]
driver.switch_to.window(new_window)






In [ ]:
scrape_exam_papers(driver, 0)

In [ ]:
scrape_exam_papers(driver, 3)

In [ ]:
scrape_exam_papers(driver, 4)

In [ ]:
scrape_exam_papers(driver, 9)

In [ ]:
scrape_exam_papers(driver, 13)

In [17]:
scrape_exam_papers(driver, 15)

Clicked 'Load More' button.
Clicked 'Load More' button.
'Show More' button not visible. Continuing...
{'name': 'Management', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316970002021&lang=en', 'course_name': 'MG100 - Leadership and Communication in Teams', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235553280002021'}
'Show More' button not visible. Continuing...
{'name': 'Management', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316970002021&lang=en', 'course_name': 'MG101 - Core Business Disciplines: Finance and Operations Management', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235553630002021'}
'Show More' button not visible. Continuing...
{'name': 'M

[{'name': 'Management',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316970002021&lang=en',
  'course_name': 'MG100 - Leadership and Communication in Teams',
  'type': 'EXAM PAPER',
  'exam_year': '2017',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235553280002021'},
 {'name': 'Management',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316970002021&lang=en',
  'course_name': 'MG101 - Core Business Disciplines: Finance and Operations Management',
  'type': 'EXAM PAPER',
  'exam_year': '2017',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235553630002021'},
 {'name': 'Management',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316970002021&lang=en',
  'c

In [18]:
scrape_exam_papers(driver, 18)

Clicked 'Show More' button.
{'name': 'Methodology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en', 'course_name': 'MY451 - Introduction to Quantitative Analysis', 'type': 'EXAM PAPER', 'exam_year': '2017', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235602510002021'}
{'name': 'Methodology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en', 'course_name': 'MY451 - Introduction to Quantitative Analysis', 'type': 'EXAM PAPER', 'exam_year': '2019', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235602530002021'}
{'name': 'Methodology', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en', 'course_name': 'MY451 -

[{'name': 'Methodology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en',
  'course_name': 'MY451 - Introduction to Quantitative Analysis',
  'type': 'EXAM PAPER',
  'exam_year': '2017',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235602510002021'},
 {'name': 'Methodology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en',
  'course_name': 'MY451 - Introduction to Quantitative Analysis',
  'type': 'EXAM PAPER',
  'exam_year': '2019',
  'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235602530002021'},
 {'name': 'Methodology',
  'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235316960002021&lang=en',
  'course_name': 'MY451 